This notebook contains combination combination of our work on the LTN. It consinsts of the most important findings and LTN implementation in a structured way. This is not all the code that was developed during the project and only the most interesting parts. More code is available on the development branch in few separate notebooks. 

# Import of libraries and data preparation

In [2]:
import torch
import pandas as pd
import ltn
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt


## Loading the dataset

In [3]:
ds_l = pd.read_csv('src\data\Stud_E-mobility_data_staticLimit.csv')

Main dataset with no labeling

In [4]:
ds_l.columns

Index(['_time', 'GARAGE_EXTERNAL_POWER', 'DEMAND_LIMIT',
       'DEMAND_LIMIT_INDICATOR', 'BATTERY_SOC', 'BATTERY_DISCHARGE_POWER',
       'BATTERY_CHARGED_ENERGY', 'BATTERY_DISCHARGED_ENERGY', 'PV_POWER',
       'PV_ENERGY', 'WALLBOX_ALPHA_ENERGY', 'WALLBOX_ALPHA_POWER',
       'WALLBOX_1_ENERGY', 'WALLBOX_1_POWER', 'WALLBOX_2_ENERGY',
       'WALLBOX_2_POWER', 'WALLBOX_3_ENERGY', 'WALLBOX_3_POWER',
       'WALLBOX_A_ENERGY', 'WALLBOX_A_POWER', 'WALLBOX_B_ENERGY',
       'WALLBOX_B_POWER', 'WALLBOX_C_ENERGY', 'WALLBOX_C_POWER',
       'WALLBOX_FASTCHARGER_ENERGY', 'WALLBOX_FASTCHARGER_POWER'],
      dtype='object')

In [5]:
ds = ds_l[['GARAGE_EXTERNAL_POWER','DEMAND_LIMIT',
       'BATTERY_SOC', 'BATTERY_DISCHARGE_POWER',
       'WALLBOX_FASTCHARGER_POWER', 'PV_POWER'
    ]]

Dataset that is independent form GT


Ground truth labeling (whole dataset)

In [16]:
gt_ds = ds.copy()

In [17]:
def label_charging(row):
    if row["BATTERY_SOC"] > 80:
        return "Fully Covered by Local Battery"
    elif 40 <= row["BATTERY_SOC"] < 80:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    elif 15 <= row["BATTERY_SOC"] <= 40:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    # elif row["BATTERY_SOC"] < 15:
    elif row["BATTERY_SOC"] < 15:
        return "Battery Discharge Stopped due to Battery Health"
    else:
        print(row["BATTERY_SOC"])
        print(row["GARAGE_EXTERNAL_POWER"])
        return "Unknown"

# Apply the labeling function to create the new column "DRAWN_FROM"
gt_ds["DRAWN_FROM"] = gt_ds.apply(label_charging, axis=1)

In [18]:
gt_ds['DRAWN_FROM'].value_counts()

DRAWN_FROM
Battery Charged from Grid                          54783
Partially Covered by Local Battery                  4457
Battery Discharge Stopped due to Battery Health      202
Name: count, dtype: int64

Small labeled dataset (only the part that follows GT)

In [9]:
delta = 0.5 # Tolerance for the power limit
SOC_less_15 = gt_ds[(gt_ds["BATTERY_SOC"]<=15) & (gt_ds["BATTERY_DISCHARGE_POWER"]<=0)]
SOC_less_40_1 = gt_ds[(gt_ds["BATTERY_SOC"]>15) &(gt_ds["BATTERY_SOC"]<40) & (gt_ds["GARAGE_EXTERNAL_POWER"]<50) & (gt_ds["BATTERY_DISCHARGE_POWER"]<0)]
SOC_less_40_2 = gt_ds[(gt_ds["BATTERY_SOC"]>15) &(gt_ds["BATTERY_SOC"]<40) & (gt_ds["GARAGE_EXTERNAL_POWER"]<=(50+delta)) & ((50-delta)<=gt_ds["GARAGE_EXTERNAL_POWER"]) & (gt_ds["BATTERY_DISCHARGE_POWER"]>=0)]
SOC_more_40 = gt_ds[(gt_ds["BATTERY_SOC"]>=40) & (gt_ds["BATTERY_DISCHARGE_POWER"]>=0)]

In [10]:
gt_ds_small = pd.concat([SOC_less_15, SOC_less_40_1, SOC_less_40_2, SOC_more_40], ignore_index=True)
gt_ds_small = gt_ds_small.drop_duplicates()
print(f"Percentage of dataset, that is kept: {len(gt_ds_small)/len(gt_ds)*100}%")

Percentage of dataset, that is kept: 16.313381110998957%


Getting features and target

In [11]:
# uncoment and use for testing Model 1 - small dataset
features = gt_ds_small.drop(['DEMAND_LIMIT','GARAGE_EXTERNAL_POWER', 'DRAWN_FROM'], axis=1)
target = gt_ds_small['DRAWN_FROM']

In [62]:
features = gt_ds.drop(['DEMAND_LIMIT','GARAGE_EXTERNAL_POWER', 'DRAWN_FROM'], axis=1)
target = gt_ds['DRAWN_FROM']

In [12]:
encoder = LabelEncoder()
en_targ = encoder.fit_transform(target)

In [13]:
# Print classes and their labels
for label, original_class in enumerate(encoder.classes_):
    print(f'Original Class: "{original_class}" is encoded as {label}')

Original Class: "Battery Charged from Grid" is encoded as 0
Original Class: "Battery Discharge Stopped due to Battery Health" is encoded as 1
Original Class: "Partially Covered by Local Battery" is encoded as 2


# Logic Tensor Network

## Simple LTN classification model

Split data into train and test, encode the labels into ltn Constants

In [65]:

features_train, features_test, target_train, target_test = train_test_split(features, en_targ, test_size=0.2, random_state=42)
features_train = torch.tensor(features_train.to_numpy()).float()
features_test = torch.tensor(features_test.to_numpy()).float()

In [66]:
l_A = ltn.Constant(torch.tensor([1, 0, 0]))
l_B = ltn.Constant(torch.tensor([0, 1, 0]))
l_C = ltn.Constant(torch.tensor([0, 0, 1]))

### Models implementation and main predicate

We need two separated models because we need both logits and probabilities. Logits are used to compute the classification accuracy, while probabilities are interpreted as truth values to compute the satisfaction level of the knowledge base.

In [67]:
class MLP(torch.nn.Module):
    def __init__(self, layer_sizes=(4, 100, 52, 52, 3)):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.2)
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x, training=False):
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
            if training:
                x = self.dropout(x)
        logits = self.linear_layers[-1](x)
        return logits

class LogitsToPredicate(torch.nn.Module):
    """
    This model has inside a logits model, that is a model which compute logits for the classes given an input example x.
    The idea of this model is to keep logits and probabilities separated. The logits model returns the logits for an example,
    while this model returns the probabilities given the logits model.

    In particular, it takes as input an example x and a class label l. It applies the logits model to x to get the logits.
    Then, it applies a softmax function to get the probabilities per classes. Finally, it returns only the probability related
    to the given class l.
    """
    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, l, training=False):
        logits = self.logits_model(x, training=training)
        probs = self.softmax(logits)
        out = torch.sum(probs * l, dim=1)
        return out


In [31]:
class DataLoader(object):
    def __init__(self,
                 data,
                 labels,
                 batch_size=1,
                 shuffle=True):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.unique_labels = np.unique(labels) 

    def __len__(self):
        return int(np.ceil(self.data.shape[0] / self.batch_size))

    def __iter__(self):
        indices_per_class = {label: np.where(self.labels == label)[0] for label in self.unique_labels}

        samples_per_class = self.batch_size // len(self.unique_labels)

        for _ in range(len(self)):
            batch_indices = []

            for label in self.unique_labels:
                class_indices = np.random.choice(indices_per_class[label], size=samples_per_class, replace=True)
                batch_indices.extend(class_indices)


            if len(batch_indices) < self.batch_size:
                extra_indices = np.random.choice(np.arange(len(self.labels)), size=self.batch_size - len(batch_indices))
                batch_indices.extend(extra_indices)

            if self.shuffle:
                np.random.shuffle(batch_indices)

            yield self.data[batch_indices], self.labels[batch_indices]

Here we use LTN to define a predicate that will make use of our model

In [69]:
mlp = MLP()
P = ltn.Predicate(LogitsToPredicate(mlp))

### Utilities used by LTN, logistic operations, satisfaction and accuracy retrival

In [35]:
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

In [71]:
def compute_sat_level(loader):
    mean_sat = 0
    for data, labels in loader:
        x_A = ltn.Variable("x_A", data[labels == 0])
        x_B = ltn.Variable("x_B", data[labels == 1])
        x_C = ltn.Variable("x_C", data[labels == 2])
        mean_sat += SatAgg(
            Forall(x_A, P(x_A, l_A)),
            Forall(x_B, P(x_B, l_B)),
            Forall(x_C, P(x_C, l_C))
        )
    mean_sat /= len(loader)
    return mean_sat

def compute_accuracy(loader):
    mean_accuracy = 0.0
    for data, labels in loader:
        predictions = mlp(data).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)

    return mean_accuracy / len(loader)


In [72]:

train_loader = DataLoader(features_train, target_train, 64, shuffle=True)
test_loader = DataLoader(features_test, target_test, 64, shuffle=False)

### Learning 

In [46]:
optimizer = torch.optim.Adam(P.parameters(), lr=0.001)

for epoch in range(120):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels == 0]) 
        x_B = ltn.Variable("x_B", data[labels == 1]) 
        x_C = ltn.Variable("x_C", data[labels == 2]) 

        # calculating the satisfaction level of the knowledge base, used in guided learning
        sat_agg = SatAgg(
            Forall(x_A, P(x_A, l_A, training=True)),
            Forall(x_B, P(x_B, l_B, training=True)),
            Forall(x_C, P(x_C, l_C, training=True))
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    # Here is an example of simple quering of the logistic formulas
    if epoch % 20 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

 epoch 0 | loss 0.2683 | Train Sat 0.770 | Test Sat 0.771 | Train Acc 0.928 | Test Acc 0.929
 epoch 20 | loss 0.2072 | Train Sat 0.792 | Test Sat 0.782 | Train Acc 0.943 | Test Acc 0.941
 epoch 40 | loss 0.2096 | Train Sat 0.793 | Test Sat 0.801 | Train Acc 0.944 | Test Acc 0.943
 epoch 60 | loss 0.2055 | Train Sat 0.799 | Test Sat 0.798 | Train Acc 0.943 | Test Acc 0.944
 epoch 80 | loss 0.2040 | Train Sat 0.795 | Test Sat 0.785 | Train Acc 0.944 | Test Acc 0.941
 epoch 100 | loss 0.2022 | Train Sat 0.802 | Test Sat 0.809 | Train Acc 0.949 | Test Acc 0.949


Final result

In [48]:
print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))

# epoch 119 | loss 0.2018 | Train Sat 0.801 | Test Sat 0.808 | Train Acc 0.948 | Test Acc 0.950

 epoch 119 | loss 0.2018 | Train Sat 0.801 | Test Sat 0.808 | Train Acc 0.948 | Test Acc 0.950


For that model already after 20 epochs we can see Test Acc == 0.941 and the rule satisfaction on Test == 0.782

## Rules satisfaction presentation
Rules satisfaction allows for a simple verification of some believes, theories as to the data (or simply verification of what we thing could be a ,,rule'' as to the system behaviour). This, to some extend, is usually possible with data analysis (in case of simple rules), but as shown later on - plays a crucial role in finding a treshold and allows for a things outside of the scope of the simple data analysis.

In [39]:
testing_set = DataLoader(torch.tensor(features.to_numpy()).float(), torch.tensor(en_targ), 64, shuffle=False)

In [40]:
def compute_sat_level_phi(loader, phi):
    sat_values = []
    for features, labels in loader:
        for feature, label in zip(features, labels):
            sat_values.append(phi(feature, label).value)
    mean_sat = torch.mean(torch.stack(sat_values))
    return mean_sat

Check the satisfaction level of the rule from the GT 
- If SOC < 15 than Battery Discharge Stopped due to Battery Health 

In [41]:
def discharge_stopped(tensor):
    return tensor <= 0

def Battery_SOC_small(tensor):
    return tensor < 15 

Battery_SOC_small = ltn.Predicate(None, Battery_SOC_small)
discharge_stopped = ltn.Predicate(None, discharge_stopped)

In [42]:

def phi(features, label):
    sc = features[0].view(-1,1)
    bp = features[1].view(-1,1)

    bp = ltn.Variable("bp", bp)
    s = ltn.Variable("s", sc)
    return Forall(s, Implies(Battery_SOC_small(s), discharge_stopped(bp)), p=5)



In [43]:
compute_sat_level_phi(testing_set, phi)

tensor(0.7488)

## Threshold optimization (3 Models mentioned in the report)

First we define a simple regression model, that will be used for finding the thresholds

In [26]:
class ThresholdModel(torch.nn.Module):
    """
    This model returns a single value (threshold) given a set of features. 
    """
    def __init__(self, input_size, hidden_size):
        super(ThresholdModel, self).__init__()
        self.elu = torch.nn.ELU()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, 1)

    def forward(self, x):
        """
        Method which defines the forward phase of the neural network.

        :param x: the features of the example
        :return: threshold for example x
        """
        x = self.elu(self.linear1(x))
        out = self.linear2(x)
        return out


In [75]:
tmodel = ltn.Function(ThresholdModel(input_size=1, hidden_size=8))
mlp = MLP()
P = ltn.Predicate(LogitsToPredicate(mlp))

Simple query to present checking rule satisfaction during training 

In [22]:
def Battery_SOC_smaller(tensor, tr):
    return tensor <= tr

Battery_SOC_smaller = ltn.Predicate(None, Battery_SOC_smaller)

Predicates used to guide the regression task 

In [76]:
Battery_SOC_smaller2 = ltn.Predicate(func=lambda tensor, tr: torch.sigmoid(tensor - tr))
Battery_SOC_larger2 = ltn.Predicate(func=lambda tensor, tr: torch.sigmoid(tr - tensor))

### Model 1 and Model 2 
- to run it corectly make sure to use the right dataset
- for Model 1 we run it on small dataset - that results in a value close to 15 for 1st threshold and close to 40 for the second one (If we comment parts of code that are used for threshold2 in sat_agg part, than the value will be closer to 15% SOC, that happens since probably the overall loss is smaller and our rules are not optimal, or lack some extra information)
- for Model 2 make sure to run it on the big dataset. Resulting thresholds should be: for 1st the threshold is much closer to 15 than for small dataset, for 2nd the value is around 44% SOC, which is in agreement with our findings from the DNN model

Generally the models work by combining the classification task and regression task together - we optimize both at the same time. The models aim to optimize rules specified in sat_agg. The classification model is present, because we do want to include our knowledge from GT model. 

In [77]:
optimizer = torch.optim.Adam(list(P.parameters()) + list(tmodel.parameters()), lr=0.001)

for epoch in range(120):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels == 0]) # class A examples
        x_B = ltn.Variable("x_B", data[labels == 1]) # class B examples
        x_C = ltn.Variable("x_C", data[labels == 2]) # class C examples

        x_A_SOC = ltn.Variable("x_A_SOC", data[labels == 0][:, 0])
        x_B_SOC = ltn.Variable("x_B_SOC", data[labels == 1][:, 0])
        x_C_SOC = ltn.Variable("x_C_SOC", data[labels == 2][:, 0])

        tr_soc = ltn.Variable("tr_soc", torch.from_numpy(np.concatenate([data[labels == 0][:, 0], data[labels == 1][:, 0]])))
        tr2_soc = ltn.Variable("tr_soc", torch.from_numpy(np.concatenate([data[labels == 0][:, 0], data[labels == 2][:, 0]])))
        tresh1 = tmodel(x_B_SOC)
        tresh2 = tmodel(x_A_SOC)
        
        tresh = ltn.Variable("tre1", tresh1.value)
        tresh2 = ltn.Variable("tre2", tresh2.value)
        
        sat_agg = SatAgg(    
            # there is a treshold such that SOC is smaller than treshold 
            Forall(x_A, P(x_A, l_A, training=True)),
            Forall(x_B, P(x_B, l_B, training=True)),
            Forall(x_C, P(x_C, l_C, training=True)),
            Exists(tresh, Forall(x_B_SOC, Battery_SOC_larger2(x_B_SOC, tresh), p=5), p=5),
            Exists(tresh, Forall(x_C_SOC, Battery_SOC_smaller2(x_C_SOC, tresh), p=5) ,p=5),
            Exists(tresh2, Forall(x_A_SOC, Battery_SOC_larger2(x_A_SOC, tresh2), p=5) ,p=5),
            Exists(tresh2, Forall(x_C_SOC, Battery_SOC_smaller2(x_C_SOC, tresh2), p=5) ,p=5),
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    if epoch % 20 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train Acc %.3f | Test Acc %.3f"
              %(epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_accuracy(train_loader), compute_accuracy(test_loader)))
        print("Lower treshold 1:", tresh.value.mean(), Forall(x_B_SOC, Battery_SOC_smaller(x_B_SOC, tresh)).value.mean())
        print("Lower treshold 2:", tresh2.value.mean(), Forall(x_A_SOC, Battery_SOC_smaller(x_A_SOC, tresh2)).value.mean())

 epoch 0 | loss 0.5540 | Train Sat 0.766 | Test Sat 0.768 | Train Acc 0.928 | Test Acc 0.934
Lower treshold 1: tensor(8.5343, grad_fn=<MeanBackward0>) tensor(0.1256)
Lower treshold 2: tensor(18.5811, grad_fn=<MeanBackward0>) tensor(0.)
 epoch 20 | loss 0.3829 | Train Sat 0.797 | Test Sat 0.791 | Train Acc 0.942 | Test Acc 0.946
Lower treshold 1: tensor(14.8571, grad_fn=<MeanBackward0>) tensor(0.7316)
Lower treshold 2: tensor(44.0128, grad_fn=<MeanBackward0>) tensor(0.7618)
 epoch 40 | loss 0.3829 | Train Sat 0.800 | Test Sat 0.797 | Train Acc 0.944 | Test Acc 0.943
Lower treshold 1: tensor(14.9607, grad_fn=<MeanBackward0>) tensor(0.9057)
Lower treshold 2: tensor(44.3005, grad_fn=<MeanBackward0>) tensor(0.7256)
 epoch 60 | loss 0.3849 | Train Sat 0.799 | Test Sat 0.803 | Train Acc 0.946 | Test Acc 0.945
Lower treshold 1: tensor(15.4450, grad_fn=<MeanBackward0>) tensor(0.8292)
Lower treshold 2: tensor(44.5021, grad_fn=<MeanBackward0>) tensor(0.5837)


KeyboardInterrupt: 

For small GT dataset

In [ ]:
#  epoch 0 | loss 0.5941 | Train Sat 0.664 | Test Sat 0.671 | Train Acc 0.841 | Test Acc 0.844
# Lower treshold 1: tensor(5.6715, grad_fn=<MeanBackward0>) tensor(0.)
# Lower treshold 2: tensor(18.6631, grad_fn=<MeanBackward0>) tensor(0.)
#  epoch 20 | loss 0.3606 | Train Sat 0.684 | Test Sat 0.678 | Train Acc 0.845 | Test Acc 0.850
# Lower treshold 1: tensor(13.8472, grad_fn=<MeanBackward0>) tensor(0.7308)
# Lower treshold 2: tensor(39.3693, grad_fn=<MeanBackward0>) tensor(0.7790)
#  epoch 40 | loss 0.3577 | Train Sat 0.686 | Test Sat 0.679 | Train Acc 0.850 | Test Acc 0.860
# Lower treshold 1: tensor(14.0685, grad_fn=<MeanBackward0>) tensor(0.7352)
# Lower treshold 2: tensor(39.5058, grad_fn=<MeanBackward0>) tensor(0.7332)
#  epoch 60 | loss 0.3599 | Train Sat 0.687 | Test Sat 0.679 | Train Acc 0.851 | Test Acc 0.840
# Lower treshold 1: tensor(13.8678, grad_fn=<MeanBackward0>) tensor(0.6503)
# Lower treshold 2: tensor(38.7538, grad_fn=<MeanBackward0>) tensor(0.5950)
#  epoch 80 | loss 0.3525 | Train Sat 0.685 | Test Sat 0.687 | Train Acc 0.849 | Test Acc 0.859
# Lower treshold 1: tensor(14.5301, grad_fn=<MeanBackward0>) tensor(0.7022)
# Lower treshold 2: tensor(39.4339, grad_fn=<MeanBackward0>) tensor(0.7049)
#  epoch 100 | loss 0.3597 | Train Sat 0.678 | Test Sat 0.685 | Train Acc 0.851 | Test Acc 0.854
# Lower treshold 1: tensor(13.4160, grad_fn=<MeanBackward0>) tensor(0.7180)
# Lower treshold 2: tensor(42.0435, grad_fn=<MeanBackward0>) tensor(0.5395)

For the big GT dataset

In [ ]:
# epoch 0 | loss 0.5540 | Train Sat 0.766 | Test Sat 0.768 | Train Acc 0.928 | Test Acc 0.934
# Lower treshold 1: tensor(8.5343, grad_fn=<MeanBackward0>) tensor(0.1256)
# Lower treshold 2: tensor(18.5811, grad_fn=<MeanBackward0>) tensor(0.)
#  epoch 20 | loss 0.3829 | Train Sat 0.797 | Test Sat 0.791 | Train Acc 0.942 | Test Acc 0.946
# Lower treshold 1: tensor(14.8571, grad_fn=<MeanBackward0>) tensor(0.7316)
# Lower treshold 2: tensor(44.0128, grad_fn=<MeanBackward0>) tensor(0.7618)
#  epoch 40 | loss 0.3829 | Train Sat 0.800 | Test Sat 0.797 | Train Acc 0.944 | Test Acc 0.943
# Lower treshold 1: tensor(14.9607, grad_fn=<MeanBackward0>) tensor(0.9057)
# Lower treshold 2: tensor(44.3005, grad_fn=<MeanBackward0>) tensor(0.7256)
#  epoch 60 | loss 0.3849 | Train Sat 0.799 | Test Sat 0.803 | Train Acc 0.946 | Test Acc 0.945
# Lower treshold 1: tensor(15.4450, grad_fn=<MeanBackward0>) tensor(0.8292)
# Lower treshold 2: tensor(44.5021, grad_fn=<MeanBackward0>) tensor(0.5837)

### Model 3 
In this task we only use the Regression model to find the best thresholds. Thanks to it we retrvie the 44% SOC. Well-defined rules enable us to find thresholds independently of GT. This leads to the conclusion that it is possible to discover other system rules by simply defining expert knowledge as a set of constraints that guide the regression task

This part creates the DS 

In [20]:
s_data = ds_l[['_time','GARAGE_EXTERNAL_POWER', 'DEMAND_LIMIT',
      #  'DEMAND_LIMIT_INDICATOR', 
       'BATTERY_SOC', 'BATTERY_DISCHARGE_POWER',
       'BATTERY_CHARGED_ENERGY','WALLBOX_FASTCHARGER_POWER', 'BATTERY_DISCHARGED_ENERGY', 'PV_POWER',
       'PV_ENERGY'
    ]]

def label_charging(row):
    if row["BATTERY_SOC"] > 80:
        return "Fully Covered by Local Battery"
    elif 40 <= row["BATTERY_SOC"] < 80:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    elif 15 <= row["BATTERY_SOC"] <= 40:
        if row["GARAGE_EXTERNAL_POWER"] > row["DEMAND_LIMIT"]:
            return "Partially Covered by Local Battery"
        else:
            return "Battery Charged from Grid"
    # elif row["BATTERY_SOC"] < 15:
    elif row["BATTERY_SOC"] < 15:
        return "Battery Discharge Stopped due to Battery Health"
    else:
        print(row["BATTERY_SOC"])
        print(row["GARAGE_EXTERNAL_POWER"])
        return "Unknown"

s_data["DRAWN_FROM"] = s_data.apply(label_charging, axis=1)


features = s_data.drop(['_time','DEMAND_LIMIT', 'PV_POWER','PV_ENERGY', 'DRAWN_FROM'], axis=1)
target = s_data['DRAWN_FROM']

encoder = LabelEncoder()
en_targ = encoder.fit_transform(target)

D:\tmp\ipykernel_25548\3397578051.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_data["DRAWN_FROM"] = s_data.apply(label_charging, axis=1)


In [21]:
# Print classes and their labels
for label, original_class in enumerate(encoder.classes_):
    print(f'Original Class: "{original_class}" is encoded as {label}')

Original Class: "Battery Charged from Grid" is encoded as 0
Original Class: "Battery Discharge Stopped due to Battery Health" is encoded as 1
Original Class: "Partially Covered by Local Battery" is encoded as 2


In [22]:

features_train, features_test, target_train, target_test = train_test_split(features, en_targ, test_size=0.2, random_state=42)
features_train = torch.tensor(features_train.to_numpy()).float()
features_test = torch.tensor(features_test.to_numpy()).float()

l_A = ltn.Constant(torch.tensor([1, 0, 0]))
l_B = ltn.Constant(torch.tensor([0, 1, 0]))
l_C = ltn.Constant(torch.tensor([0, 0, 1]))


In [27]:
tmodel = ltn.Function(ThresholdModel(input_size=1, hidden_size=8))


Make sure that you run DataLoader from the 1st part of the notebook

In [32]:
train_loader = DataLoader(features_train, target_train, 64, shuffle=True)
test_loader = DataLoader(features_test, target_test, 64, shuffle=False)

Predicates

In [37]:
def BDP_neg(tensor):
    return tensor <= 0

BDP_neg = ltn.Predicate(None, BDP_neg)

def BDP_p(tensor):
    return tensor >= 0

BDP_p = ltn.Predicate(None, BDP_p)

def gep_under(tensor):
    return tensor < 50

gep_under = ltn.Predicate(None, gep_under)
def gep_abov(tensor):
    return tensor >= 50

gep_abov = ltn.Predicate(None, gep_abov)

Battery_SOC_smaller2 = ltn.Predicate(func=lambda tensor, tr: torch.sigmoid(tensor - tr))
Battery_SOC_larger2 = ltn.Predicate(func=lambda tensor, tr: torch.sigmoid(tr - tensor))

In [ ]:
optimizer = torch.optim.Adam(list(tmodel.parameters()), lr=0.001)


for epoch in range(120):
    train_loss = 0.0
    for batch_idx, (data, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # we ground the variables with current batch data
        x_A = ltn.Variable("x_A", data[labels == 0]) # class A examples
        x_B = ltn.Variable("x_B", data[labels == 1]) # class B examples
        x_C = ltn.Variable("x_C", data[labels == 2]) # class C examples

        x = ltn.Variable("x", data)
        gep = ltn.Variable("gep", data[:, 0])
        soc = ltn.Variable("soc", data[:, 1])
        bce = ltn.Variable("bce", data[:, 3])
        bde = ltn.Variable("bde", data[:, 5])

        x_B_SOC = ltn.Variable("x_B_SOC", data[labels == 1][:, 0])

        x_C_SOC = ltn.Variable("x_C_SOC", data[labels == 2][:, 0])
        x_A_SOC = ltn.Variable("x_A_SOC", data[labels == 0][:, 0])

        tresh1 = tmodel(soc)
        tresh2 = tmodel(soc)
        tresh = ltn.Variable("tre1", tresh1.value)
        tresh2 = ltn.Variable("tre2", tresh2.value)
        # print(Exists(tresh, Forall(soc, And(BDP_neg(bdp), Battery_SOC_smaller2(soc, tresh)), p=5), p=5))
        sat_agg = SatAgg(    
            Exists(tresh, Forall(soc, Implies(And(gep_under(gep), BDP_p(bde)), Battery_SOC_larger2(soc, tresh)), p=5), p=5).value.mean(),
            Exists(tresh, Forall(soc, Implies(And(gep_under(gep), BDP_p(bce)), Battery_SOC_smaller2(soc, tresh)), p=5), p=5).value.mean(),
            Exists(tresh2, Forall(soc, Implies(And(gep_under(gep), BDP_p(bde)), Battery_SOC_larger2(soc, tresh2)), p=5), p=5).value.mean(),
            Exists(tresh2, Forall(soc, Implies(And(gep_abov(gep), BDP_p(bce)), Battery_SOC_smaller2(soc, tresh2)), p=5), p=5).value.mean(),
        )
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 20 epochs of training
    if epoch % 1 == 0:
        print(" epoch %d | loss %.4f"
              %(epoch, train_loss))
        print("Lower treshold 1:", tresh.value.mean(), Forall(x_A_SOC, Battery_SOC_smaller(x_A_SOC, tresh)).value.mean())
        print("Lower treshold 2:", tresh2.value.mean(), Forall(x_A_SOC, Battery_SOC_smaller(x_A_SOC, tresh2)).value.mean())
 